# CSS 300 | Spring 2025 | Lab Activity 5 | Data Cleaning and EDA

*The very first thing you should do: rename this file by replacing "YOURNAME" with your actual first name. For this Lab Activity, this is the only file that you should submit!*

If you find yourself uncertain about how to do something, you should (in order):

- Have a look at the [`pandas` API reference](https://pandas.pydata.org/docs/reference/index.html)
- Ask Lucas or a Learning Support Specialist for help

*You are reminded that the use of generative AI in CSS 300, in any shape or form, is considered academic dishonesty and will result in a grade of zero (and possibly worse!).*

In [1]:
# Don't edit this cell, but please make sure you run it!
import pandas as pd

<hr style="border: 5px solid #0072CE;" />

For this section, we'll use the `elections` `DataFrame` that we've discussed in class. Let's read it in:

In [ ]:
elections = pd.read_csv("data/elections.csv")
elections.head(5)

---

### Question 1a
Using `groupby.agg` or one of the shorthand methods (`groupby.min`, `groupby.first`, etc.), create a `Series` object `best_result_percentage_only` that returns a `Series` showing the entire best result for every party, sorted in decreasing order. Your `Series` should include only parties that have earned at least 10% of the vote in some election. Your result should look something like this:

<code>
Party
Democratic               61.344703
Republican               60.907806
Democratic-Republican    57.210122
National Union           54.951512
Whig                     53.051213
Liberal Republican       44.071406
National Republican      43.796073
Northern Democratic      29.522311
Progressive              27.457433
American                 21.554001
Independent              18.956298
Southern Democratic      18.138998
American Independent     13.571218
Constitutional Union     12.639283
Free Soil                10.138474
Name: %, dtype: float64
</code>
<br/>

A list of named `groupby.agg` shorthand methods are [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation) (you'll have to scroll down about one page).

In [ ]:
elections_higher_than_10_percent = elections[elections['%']>= 10]

best_result_percentage_only = elections_higher_than_10_percent.groupby('Party')['%'].max().sort_values(ascending=False)

print(best_result_percentage_only)

---

### Question 1b

Repeat Question 1a. However, this time, your result should be a `DataFrame` showing all available information (all columns) rather than only the percentage as a `Series`.

This question is trickier than Question 1a. Make sure to check the lecture slides if you're stuck! It's very easy to make a subtle mistake that shows Woodrow Wilson and William Taft both winning the 2020 election.

For example, the first 3 rows of your table should be:

|Party | Year | Candidate      | Popular Vote | Result | %         |
|------|------|----------------|--------------|--------|-----------|
|**Democratic**  | 1964 | Lyndon Johnson | 43127041      | win   | 61.344703 |
|**Republican**  | 1972 | Richard Nixon | 47168710      | win   | 60.907806 |
|**Democratic-Republican**  | 1824 | Andrew Jackson | 151271      | loss   | 57.210122 |

Note that the index should be `Party`. In other words, don't use `reset_index`.

In [ ]:
elections_sorted_by_percent = elections_higher_than_10_percent.sort_values('%', ascending=False)

elections_sorted_by_percent.groupby('Party').first().sort_values('%', ascending=False)

---

### Question 1c

Using `filter`, create a `DataFrame` object `major_party_results_since_1988` that includes all election results starting after 1988 (exclusive) but only shows a row if the Party it belongs to has earned at least 1% of the popular vote in ANY election since 1988.

For example, despite having candidates in 2004, 2008, and 2016, no Constitution party candidates should be included since this party has not earned 1% of the vote in any election since 1988. However, you should include the Reform candidate from 2000 (Pat Buchanan) despite only having 0.43% of the vote, because in 1996, the Reform candidate Ross Perot exceeded 1% of the vote.

For example, the first three rows of the table you generate should look like:

|     |   Year | Candidate         | Party       |   Popular vote | Result   |         % |
|----:|-------:|:------------------|:------------|---------------:|:---------|----------:|
| 139 |   1992 | Andre Marrou      | Libertarian |       290087   | loss     | 0.278516  |
| 140 |   1992 | Bill Clinton      | Democratic  |       44909806 | win      | 43.118485 |
| 142 |   1992 | George H. W. Bush | Republican  |       39104550 | loss     |  37.544784|

*Hint*: The following questions might help you construct your solution. One of the lines should be identical to the `filter` examples shown above.

1) How can we **only** keep rows in the data starting after 1988 (exclusive)?
2) What column should we `groupby` to filter out parties that have earned at least 1% of the popular vote in ANY election since 1988?
3) How can we write an aggregation function that takes a sub-DataFrame and returns whether at least 1% of the vote has been earned in that sub-DataFrame? This may give you a hint about the second question!

In [ ]:
elections_since_1988 = elections[elections['Year'] > 1988]

def earned_at_least_1_percent(group):
    return (group['%'] >= 1).any()

major_party_results_since_1988 = elections_since_1988.groupby('Party').filter(earned_at_least_1_percent)

print(major_party_results_since_1988)

---

### Question 2

Using `.str.split`, create a new `DataFrame` called `elections_with_first_name` with a new column `First Name` that is equal to the Candidate's first name.

See the `pandas` `str` [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html) for documentation on using `str.split`.

In [ ]:
elections_with_first_name = elections.copy()

elections_with_first_name['First Name'] = elections['Candidate'].str.split().str[0]

elections_with_first_name.head(10)

---

### Question 3

Recall that, in lecture, we made extensive use of a `DataFrame` called `babynames`. We'll grab just the 2022 names below:

In [ ]:
# Please don't edit or amend this code!
column_labels = ['State', 'Sex', 'Year', 'Name', 'Count']
babynames = pd.read_csv("data/STATE.IL.TXT", names=column_labels)
babynames_2022 = babynames[babynames["Year"] == 2022]
babynames_2022.head()

Using the [pd.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) function described in the lecture, combine the `babynames_2022` table with the `elections_with_first_name` table you created earlier to form `presidential_candidates_and_name_popularity`. Your resulting `DataFrame` should contain all columns from both of the tables.

In [ ]:
presidential_candidates_and_name_popularity = elections_with_first_name.merge(right = babynames_2022, left_on = 'First Name', right_on = 'Name')
presidential_candidates_and_name_popularity.head(10)

---

### Question 4

Using `presidential_candidates_and_name_popularity`, create a new `DataFrame` called `party_result_popular_vote_pivot`, whose index is the `Party` and whose columns are their `Result`. Each cell should contain the total number of popular votes received. The `pandas` `pivot_table` documentation can be found [here](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

You may notice that there are `NaN`s in your table from missing data. Replace the `NaN` values with 0. You may find `.pivot_table`'s `fill_value=` argument helpful. Or, you can use `pd.DataFrame.fillna` [(documentation here)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html).

In [ ]:
import numpy as np

party_result_popular_vote_pivot = presidential_candidates_and_name_popularity.pivot_table(
    index = 'Party',
    columns = 'Result',
    values = ['Popular vote'],
    aggfunc = np.sum,
    fill_value='0'
)

party_result_popular_vote_pivot

<hr style="border: 5px solid #0072CE;" />

# Woohoo! You're all done.